In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score, mean_absolute_error,confusion_matrix,classification_report
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
!pip install scikit-multilearn
from sklearn.multioutput import ClassifierChain
from skmultilearn.problem_transform import LabelPowerset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pickle

In [4]:
df1=pd.read_csv('content/semi_processed_5.csv')

In [5]:
df1.columns

Index(['ActionPerformed', 'ConnectionClassification', 'Priority', 'Date-Time',
       'SourceIP', 'SourcePort', 'DestinationIP', 'DestinationPort',
       'TimeToLive', 'TypeOfService', 'DataGramLength', 'AllFlags', 'Seq',
       'Ack', 'Win', 'TCP_length', 'TCP_Options', 'NOP_NOP_TS', 'MSS',
       'NOP_WS'],
      dtype='object')

In [6]:
df1.drop(['Date-Time','Seq',
       'Ack', 'Win',], axis=1, inplace=True)

In [7]:
df1['ActionPerformed'] = df1['ActionPerformed'].str.replace('<', '') 

In [8]:
df1.columns = df1.columns.str.replace(' ', '') 

In [9]:
df=pd.get_dummies(data=df1, columns=['ActionPerformed', 'Priority',
       'SourceIP','DestinationIP', 'DestinationPort',
       'TimeToLive', 'TypeOfService', 'DataGramLength', 'AllFlags', 'TCP_length', 'TCP_Options','MSS',
       'NOP_WS'], drop_first=True)

In [10]:
df.columns

Index(['ConnectionClassification', 'SourcePort', 'NOP_NOP_TS',
       'ActionPerformed_ (http_inspect) BARE BYTE UNICODE ENCODING  ',
       'ActionPerformed_ (http_inspect) DOUBLE DECODING ATTACK  ',
       'ActionPerformed_ (http_inspect) IIS UNICODE CODEPOINT ENCODING  ',
       'ActionPerformed_ (http_inspect) OVERSIZE CHUNK ENCODING  ',
       'ActionPerformed_ (http_inspect) OVERSIZE REQUEST-URI DIRECTORY  ',
       'ActionPerformed_ (http_inspect) U ENCODING  ',
       'ActionPerformed_ (http_inspect) WEBROOT DIRECTORY TRAVERSAL  ',
       ...
       'TCP_length_28.0', 'TCP_length_32.0', 'TCP_length_40.0',
       'TCP_Options_1.0', 'TCP_Options_3.0', 'TCP_Options_5.0', 'MSS_1160',
       'MSS_1460', 'NOP_WS_3', 'NOP_WS_6'],
      dtype='object', length=2662)

In [11]:
df.to_csv("final_data.csv", index=False)

In [35]:
from sklearn.model_selection import train_test_split

In [36]:
X=df.drop(['ConnectionClassification'], axis=1)
y=df['ConnectionClassification']

In [37]:
X.columns

Index(['SourcePort', 'NOP_NOP_TS',
       'ActionPerformed_b'(http_inspect)BAREBYTEUNICODEENCODING'',
       'ActionPerformed_b'(http_inspect)DOUBLEDECODINGATTACK'',
       'ActionPerformed_b'(http_inspect)IISUNICODECODEPOINTENCODING'',
       'ActionPerformed_b'(http_inspect)OVERSIZECHUNKENCODING'',
       'ActionPerformed_b'(http_inspect)OVERSIZEREQUEST-URIDIRECTORY'',
       'ActionPerformed_b'(http_inspect)UENCODING'',
       'ActionPerformed_b'(http_inspect)WEBROOTDIRECTORYTRAVERSAL'',
       'ActionPerformed_b'(smtp)Attemptedheadernamebufferoverflow:121charsbeforecolon'',
       ...
       'TCP_length_28.0', 'TCP_length_32.0', 'TCP_length_40.0',
       'TCP_Options_1.0', 'TCP_Options_3.0', 'TCP_Options_5.0', 'MSS_1160',
       'MSS_1460', 'NOP_WS_3', 'NOP_WS_6'],
      dtype='object', length=2660)

Splitting the dataset into training and testing dataset

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

### XGBoost Classfier

In [39]:
import time
from xgboost import XGBClassifier

In [42]:
xgbmodel = XGBClassifier(tree_method = "gpu_hist",early_stopping=10,n_estimators=500, random_state=0)

In [43]:
# fit the model
%time xgbmodel.fit(X_train, y_train)

CPU times: user 5.52 s, sys: 522 ms, total: 6.04 s
Wall time: 6.02 s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, early_stopping=10,
              gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, tree_method='gpu_hist', verbosity=1)

In [45]:
# XGB model assessment
pred_test_xgb = xgbmodel.predict(X_test)
pred_train_xgb = xgbmodel.predict(X_train)

print(classification_report(y_test, pred_test_xgb))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1666
           1       1.00      1.00      1.00      4157

    accuracy                           1.00      5823
   macro avg       1.00      1.00      1.00      5823
weighted avg       1.00      1.00      1.00      5823



In [46]:
print('Train Accuracy: ', accuracy_score(y_train, pred_train_xgb))
print('Test Accuraccy: ', accuracy_score(y_test, pred_test_xgb))

Train Accuracy:  1.0
Test Accuraccy:  1.0


In [50]:
# save the XGB model to disk
filename = 'xgb_model.pkl'
pickle.dump(xgbmodel, open(filename, 'wb'))

### Random Forest Classifier

In [53]:
rfcmodel = RandomForestClassifier(n_estimators=100)

In [54]:
rfcmodel.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [55]:
# RFC model assessment
pred_test_rfc = rfcmodel.predict(X_test)
pred_train_rfc = rfcmodel.predict(X_train)

print(classification_report(y_test,pred_test_rfc))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1666
           1       1.00      1.00      1.00      4157

    accuracy                           1.00      5823
   macro avg       1.00      1.00      1.00      5823
weighted avg       1.00      1.00      1.00      5823



In [56]:
print('Train Accuracy: ', accuracy_score(y_train, pred_train_rfc))
print('Test Accuraccy: ', accuracy_score(y_test, pred_test_rfc))

Train Accuracy:  1.0
Test Accuraccy:  1.0


In [61]:
# save the RFC model to disk
filename = 'rfc_model.pkl'
pickle.dump(rfcmodel, open(filename, 'wb'))

### Adaboost Classifier

In [57]:
adbmodel = AdaBoostClassifier(n_estimators=100)

In [58]:
adbmodel.fit(X_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=100, random_state=None)

In [59]:
# ADB model assessment
pred_test_adb = ada.predict(X_test)
pred_train_adb = ada.predict(X_train)

print(classification_report(y_test,pred_test_adb))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1666
           1       1.00      1.00      1.00      4157

    accuracy                           1.00      5823
   macro avg       1.00      1.00      1.00      5823
weighted avg       1.00      1.00      1.00      5823



In [60]:
print('Train Accuracy: ', accuracy_score(y_train, pred_train_adb))
print('Test Accuraccy: ', accuracy_score(y_test, pred_test_adb))

Train Accuracy:  1.0
Test Accuraccy:  1.0


In [62]:
# save the ADB model to disk
filename = 'adb_model.pkl'
pickle.dump(adbmodel, open(filename, 'wb'))